<a href="https://colab.research.google.com/github/Ultimo15/Gu-charo-Activo-Predictor/blob/main/Guacharo_PRO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Script Forense para analizar Cóndor Gana y buscar datos pre-cargados
# Ejecutar en Google Colab

!pip install selenium webdriver-manager beautifulsoup4 requests
!apt-get update
!apt-get install chromium-chromedriver

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import json
import re
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta

# Configurar Chrome
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    # Removing performance logging options as they seem to cause issues
    # chrome_options.add_argument('--enable-logging')
    # chrome_options.add_argument('--log-level=0')
    # chrome_options.add_experimental_option('perfLoggingPrefs', {
    #     'enableNetwork': True,
    #     'enablePage': False,
    # })
    # chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

    driver = webdriver.Chrome(options=chrome_options)
    return driver

print("🔍 ANÁLISIS FORENSE DE CÓNDOR GANA")
print("=" * 50)

# 1. ANÁLISIS DEL CÓDIGO FUENTE HTML
print("\n1️⃣ ANALIZANDO CÓDIGO FUENTE HTML...")
driver = setup_driver()
driver.get("https://tripletachira.com/condor.php")
time.sleep(3)

html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')

# Buscar datos en comentarios HTML
print("\n📝 Buscando en comentarios HTML...")
comments = soup.find_all(string=lambda text: isinstance(text, str) and text.strip().startswith('<!--'))
for comment in comments:
    if any(keyword in comment.lower() for keyword in ['resultado', 'numero', 'ganador', 'premio', 'sorteo']):
        print(f"⚠️ Comentario sospechoso encontrado: {comment[:100]}...")

# 2. ANÁLISIS DE SCRIPTS JAVASCRIPT
print("\n2️⃣ ANALIZANDO SCRIPTS JAVASCRIPT...")
scripts = soup.find_all('script')
suspicious_patterns = []

for i, script in enumerate(scripts):
    if script.string:
        content = script.string

        # Buscar arrays o objetos con datos
        array_pattern = r'(?:var|let|const)\s+(\w+)\s*=\s*\[(.*?)\]'
        object_pattern = r'(?:var|let|const)\s+(\w+)\s*=\s*\{(.*?)\}'

        arrays = re.findall(array_pattern, content, re.DOTALL)
        objects = re.findall(object_pattern, content, re.DOTALL)

        # Buscar fechas futuras
        future_dates = []
        date_patterns = [
            r'\d{4}-\d{2}-\d{2}',
            r'\d{2}/\d{2}/\d{4}',
            r'\d{2}-\d{2}-\d{4}'
        ]

        for pattern in date_patterns:
            dates = re.findall(pattern, content)
            for date_str in dates:
                try:
                    # Intentar parsear la fecha
                    if '-' in date_str and len(date_str.split('-')[0]) == 4:
                        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
                    elif '/' in date_str:
                        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
                    else:
                        date_obj = datetime.strptime(date_str, '%d-%m-%Y')

                    if date_obj > datetime.now():
                        future_dates.append(date_str)
                except:
                    pass

        if future_dates:
            print(f"\n⚠️ Script #{i} contiene fechas futuras: {future_dates}")

        # Buscar números que podrían ser resultados
        number_patterns = re.findall(r'\b\d{2,3}\b', content)
        if len(number_patterns) > 5:
            print(f"\n🔢 Script #{i} contiene múltiples números: {number_patterns[:10]}...")

        # Buscar palabras clave
        keywords = ['resultado', 'ganador', 'premio', 'numero', 'sorteo', 'condor', 'futuro', 'proximo']
        for keyword in keywords:
            if keyword in content.lower():
                context = content[max(0, content.lower().find(keyword)-50):content.lower().find(keyword)+50]
                print(f"\n🔑 Palabra clave '{keyword}' encontrada en Script #{i}:")
                print(f"   Contexto: ...{context}...")

# 3. ANÁLISIS DE ALMACENAMIENTO LOCAL
print("\n3️⃣ ANALIZANDO ALMACENAMIENTO LOCAL...")
storage_check = """
var results = {
    localStorage: {},
    sessionStorage: {},
    cookies: document.cookie,
    indexedDB: []
};

// Revisar localStorage
for(var i = 0; i < localStorage.length; i++) {
    var key = localStorage.key(i);
    results.localStorage[key] = localStorage.getItem(key);
}

// Revisar sessionStorage
for(var i = 0; i < sessionStorage.length; i++) {
    var key = sessionStorage.key(i);
    results.sessionStorage[key] = sessionStorage.getItem(key);
}

return results;
"""

storage_data = driver.execute_script(storage_check)
print("\n📦 Datos en almacenamiento local:")
print(f"   - localStorage: {storage_data['localStorage']}")
print(f"   - sessionStorage: {storage_data['sessionStorage']}")
print(f"   - Cookies: {storage_data['cookies']}")

# 4. ANÁLISIS DE PETICIONES DE RED
print("\n4️⃣ ANALIZANDO PETICIONES DE RED...")
# Removing the problematic get_log call here
# logs = driver.get_log('performance')
network_requests = []

# The previous logs could not be retrieved, skipping the analysis of network requests from logs.
# for log in logs:
#     try:
#         message = json.loads(log['message'])
#         if 'Network.requestWillBeSent' in message['message']['method']:
#             request = message['message']['params']['request']
#             url = request['url']

#             # Filtrar URLs interesantes
#             if any(keyword in url.lower() for keyword in ['api', 'ajax', 'data', 'result', 'json', 'condor']):
#                 network_requests.append({
#                     'url': url,
#                     'method': request['method'],
#                     'headers': request.get('headers', {})
#                 })
#     except:
#         pass

print("\n🌐 Peticiones de red interesting: (Skipped due to logging issue)")
# for req in network_requests:
#     print(f"   - {req['method']} {req['url']}")


# 5. BUSCAR DATOS EN ATRIBUTOS HTML
print("\n5️⃣ BUSCANDO DATOS EN ATRIBUTOS HTML...")
elements_with_data = soup.find_all(attrs={"data-resultado": True})
elements_with_data.extend(soup.find_all(attrs={"data-number": True}))
elements_with_data.extend(soup.find_all(attrs={"data-premio": True}))

for elem in elements_with_data:
    print(f"\n⚠️ Elemento con datos: {elem.name}")
    for attr, value in elem.attrs.items():
        if 'data' in attr:
            print(f"   - {attr}: {value}")

# 6. ANÁLISIS DE VARIABLES GLOBALES JS
print("\n6️⃣ ANALIZANDO VARIABLES GLOBALES JAVASCRIPT...")
global_vars = """
var suspicious = [];
for(var prop in window) {
    if(window.hasOwnProperty(prop)) {
        var value = window[prop];
        if(prop.toLowerCase().includes('result') ||
           prop.toLowerCase().includes('ganador') ||
           prop.toLowerCase().includes('numero') ||
           prop.toLowerCase().includes('condor') ||
           prop.toLowerCase().includes('premio')) {
            suspicious.push({name: prop, value: value});
        }
    }
}
return suspicious;
"""

suspicious_globals = driver.execute_script(global_vars)
if suspicious_globals:
    print("\n🚨 Variables globales sospechosas encontradas:")
    for var in suspicious_globals:
        print(f"   - {var['name']}: {str(var['value'])[:100]}...")

# 7. BUSCAR ENDPOINTS OCULTOS
print("\n7️⃣ BUSCANDO ENDPOINTS Y APIs OCULTAS...")
# Buscar en el código fuente
api_patterns = [
    r'["\']([^"\']*api[^"\']*)["\']',
    r'["\']([^"\']*\.php[^"\']*)["\']',
    r'["\']([^"\']*\.json[^"\']*)["\']',
    r'url\s*:\s*["\']([^"\']+)["\']',
    r'fetch\s*\(["\']([^"\']+)["\']'
]

found_endpoints = set()
for pattern in api_patterns:
    matches = re.findall(pattern, html_source, re.IGNORECASE)
    found_endpoints.update(matches)

print("\n🔗 Posibles endpoints encontrados:")
for endpoint in found_endpoints:
    if not any(skip in endpoint for skip in ['google', 'facebook', 'jquery', 'bootstrap']):
        print(f"   - {endpoint}")

# 8. INTENTAR ACCEDER A ENDPOINTS COMUNES
print("\n8️⃣ PROBANDO ENDPOINTS COMUNES...")
base_url = "https://tripletachira.com/"
common_endpoints = [
    "api/condor",
    "api/resultados",
    "api/condor/resultados",
    "data/condor.json",
    "resultados/condor.json",
    "condor_data.php",
    "get_resultados.php",
    "ajax/condor.php"
]

for endpoint in common_endpoints:
    try:
        response = requests.get(base_url + endpoint, timeout=5)
        if response.status_code == 200:
            print(f"\n✅ ENDPOINT ACTIVO: {endpoint}")
            print(f"   Respuesta: {response.text[:200]}...")
    except:
        pass

# 9. ANÁLISIS FINAL
print("\n" + "="*50)
print("📊 RESUMEN DEL ANÁLISIS:")
print("="*50)

# Removed the problematic line attempting to get console data
# console_data = driver.execute_script("return window.performance.getEntries();")
# print(f"\n📈 Total de recursos cargados: {len(console_data)}")

# Buscar patrones numéricos sospechosos en todo el HTML
all_numbers = re.findall(r'\b\d{2,3}\b', html_source)
number_groups = {}
for num in all_numbers:
    if num not in number_groups:
        number_groups[num] = 0
    number_groups[num] += 1

print("\n🔢 Números más frecuentes en la página:")
sorted_numbers = sorted(number_groups.items(), key=lambda x: x[1], reverse=True)
for num, count in sorted_numbers[:10]:
    if count > 1:
        print(f"   - {num}: aparece {count} veces")

driver.quit()

print("\n✅ Análisis completado.")
print("\n💡 RECOMENDACIONES:")
print("1. Revisa los endpoints encontrados manualmente")
print("2. Los datos podrían estar cifrados en las variables JavaScript")
print("3. Monitorea las peticiones de red al cambiar fechas (if possible with a different logging setup)")
print("4. Algunos resultados podrían cargarse dinámicamente via WebSocket")

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 129 kB in 1s (117 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done

In [ ]:
# Extractor de Números y Animalitos - Cóndor Gana
# Para Google Colab - Basado en los hallazgos encontrados

!pip install selenium webdriver-manager beautifulsoup4 requests pandas
!apt-get update
!apt-get install chromium-chromedriver

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import json
import re
from datetime import datetime
from collections import defaultdict
import pandas as pd
from bs4 import BeautifulSoup
import time

print("🎯 EXTRACTOR DE NÚMEROS Y ANIMALITOS - CÓNDOR GANA")
print("Fechas objetivo: 20 y 21 de julio 2025")
print("=" * 70)

# Lista de animalitos de lotería venezolana (00-36)
ANIMALITOS = {
    "00": "Ballena", "01": "Carnero", "02": "Toro", "03": "Ciempiés", "04": "Alacrán",
    "05": "León", "06": "Rana", "07": "Perico", "08": "Ratón", "09": "Águila",
    "10": "Tigre", "11": "Gato", "12": "Caballo", "13": "Mono", "14": "Paloma",
    "15": "Zorro", "16": "Oso", "17": "Pavo", "18": "Burro", "19": "Chivo",
    "20": "Cochino", "21": "Gallo", "22": "Camello", "23": "Cebra", "24": "Iguana",
    "25": "Gallina", "26": "Vaca", "27": "Perro", "28": "Zamuro", "29": "Elefante",
    "30": "Caimán", "31": "Lapa", "32": "Ardilla", "33": "Pescado", "34": "Venado",
    "35": "Jirafa", "36": "Culebra"
}

# Configurar Chrome
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    return webdriver.Chrome(options=chrome_options)

# URLs a analizar
URLS = [
    "https://tripletachira.com/condor.php",
    "https://tripletachira.com/nadacondor.php",
    "https://tripletachira.com/sorteo.php",
    "https://tripletachira.com/resultados.php"
]

driver = setup_driver()
all_results = []

print("\n1️⃣ BUSCANDO DATOS JSON EN SCRIPTS...")

for url in URLS:
    print(f"\n📡 Analizando: {url}")

    try:
        driver.get(url)
        time.sleep(3)

        # Obtener todo el HTML
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # Buscar en todos los scripts
        scripts = soup.find_all('script')

        for i, script in enumerate(scripts):
            if script.string:
                content = script.string

                # Buscar patrones JSON con estructura de resultados
                # Patrón 1: Arrays JSON completos
                json_arrays = re.findall(r'\[({[^}]+}(?:,{[^}]+})*)\]', content)

                for json_str in json_arrays:
                    try:
                        # Agregar corchetes para parsear como array
                        data = json.loads(f'[{json_str}]')

                        for item in data:
                            if 'f' in item and ('2025-07-20' in item['f'] or '2025-07-21' in item['f']):
                                result = {
                                    'fecha': item.get('f'),
                                    'id': item.get('id', ''),
                                    'numero': item.get('N', ''),
                                    'animalito': item.get('S', ''),
                                    'url': url,
                                    'script': i
                                }
                                all_results.append(result)
                                print(f"   ✅ Encontrado: {result['fecha']} - #{result['numero']} {result['animalito']}")
                    except:
                        pass

                # Patrón 2: Objetos individuales
                individual_objects = re.findall(r'{[^{}]*"f"\s*:\s*"2025-07-2[01]"[^{}]*}', content)

                for obj_str in individual_objects:
                    try:
                        obj = json.loads(obj_str)
                        if 'N' in obj or 'S' in obj:
                            result = {
                                'fecha': obj.get('f'),
                                'id': obj.get('id', ''),
                                'numero': obj.get('N', ''),
                                'animalito': obj.get('S', ''),
                                'url': url,
                                'script': i
                            }
                            all_results.append(result)
                            print(f"   ✅ Encontrado: {result['fecha']} - #{result['numero']} {result['animalito']}")
                    except:
                        pass

                # Buscar menciones de animalitos
                for num, animal in ANIMALITOS.items():
                    if animal.lower() in content.lower():
                        # Buscar si hay fecha cerca
                        context = content[max(0, content.lower().find(animal.lower())-100):
                                         content.lower().find(animal.lower())+100]

                        dates = re.findall(r'2025-07-2[01]', context)
                        if dates:
                            print(f"   🐾 Animalito encontrado: {animal} (#{num}) - Fecha: {dates[0]}")

    except Exception as e:
        print(f"   ❌ Error: {str(e)}")

print("\n2️⃣ EJECUTANDO JAVASCRIPT PARA BUSCAR DATOS...")

# JavaScript para buscar datos en variables globales
js_search = """
var results = [];

// Buscar en window
for(var prop in window) {
    if(window.hasOwnProperty(prop)) {
        try {
            var value = window[prop];
            if(typeof value === 'object' && value !== null) {
                var str = JSON.stringify(value);
                if(str.includes('2025-07-20') || str.includes('2025-07-21')) {
                    results.push({
                        property: prop,
                        data: value
                    });
                }
            }
        } catch(e) {}
    }
}

// Buscar elementos con datos
document.querySelectorAll('[data-fecha], [data-numero], [data-animal]').forEach(el => {
    results.push({
        element: el.tagName,
        fecha: el.getAttribute('data-fecha'),
        numero: el.getAttribute('data-numero'),
        animal: el.getAttribute('data-animal')
    });
});

return results;
"""

for url in URLS[:2]:  # Solo las primeras 2 para no demorar mucho
    try:
        driver.get(url)
        time.sleep(2)

        js_results = driver.execute_script(js_search)
        if js_results:
            print(f"\n🔍 Datos JavaScript en {url}:")
            for res in js_results:
                print(f"   {res}")
    except:
        pass

print("\n3️⃣ ANÁLISIS ESPECÍFICO POR FECHA Y HORA...")

# Organizar resultados por fecha
results_by_date = defaultdict(list)
for result in all_results:
    if result['fecha']:
        results_by_date[result['fecha']].append(result)

# Buscar patrones por hora (basado en el ID)
hours = ["09", "10", "11", "12", "13", "14", "15", "16", "17", "18"]

print("\n📅 RESULTADOS ORGANIZADOS POR FECHA Y HORA:")
print("-" * 70)

for date in ["2025-07-20", "2025-07-21"]:
    print(f"\n📆 {date}:")

    date_results = results_by_date[date]

    if date_results:
        # Organizar por hora si es posible
        by_hour = defaultdict(list)

        for result in date_results:
            # Intentar extraer hora del ID
            id_str = result['id']
            for hour in hours:
                if hour in id_str:
                    by_hour[f"{hour}:00"].append(result)
                    break
            else:
                by_hour['Sin hora'].append(result)

        # Mostrar por hora
        for hour in sorted(by_hour.keys()):
            if by_hour[hour]:
                print(f"\n   🕐 {hour}:")
                for res in by_hour[hour]:
                    print(f"      → Número: {res['numero']} - {res['animalito']}")
    else:
        print("   ⚠️ No se encontraron resultados pre-cargados")

# 4. ANÁLISIS DE FRECUENCIAS
print("\n4️⃣ ANÁLISIS DE FRECUENCIAS...")

number_freq = defaultdict(int)
animal_freq = defaultdict(int)

for result in all_results:
    if result['numero']:
        number_freq[result['numero']] += 1
    if result['animalito']:
        animal_freq[result['animalito']] += 1

if number_freq:
    print("\n🔢 NÚMEROS MÁS FRECUENTES:")
    for num, count in sorted(number_freq.items(), key=lambda x: x[1], reverse=True)[:10]:
        animal = ANIMALITOS.get(num, "Desconocido")
        print(f"   {num} ({animal}): {count} veces")

if animal_freq:
    print("\n🐾 ANIMALITOS MÁS FRECUENTES:")
    for animal, count in sorted(animal_freq.items(), key=lambda x: x[1], reverse=True)[:10]:
        # Buscar número correspondiente
        num = next((k for k, v in ANIMALITOS.items() if v == animal), "??")
        print(f"   {animal} (#{num}): {count} veces")

# 5. GENERAR TABLA DE RESULTADOS
print("\n5️⃣ GENERANDO TABLA DE RESULTADOS...")

if all_results:
    df = pd.DataFrame(all_results)

    # Guardar en CSV
    df.to_csv('condor_resultados_animalitos.csv', index=False)
    print("\n📁 Resultados guardados en 'condor_resultados_animalitos.csv'")

    # Mostrar resumen
    print("\n📊 RESUMEN DE HALLAZGOS:")
    print(f"   - Total de resultados encontrados: {len(all_results)}")
    print(f"   - Resultados para 20/07/2025: {len(results_by_date['2025-07-20'])}")
    print(f"   - Resultados para 21/07/2025: {len(results_by_date['2025-07-21'])}")

driver.quit()

# RESUMEN FINAL
print("\n" + "="*70)
print("🎯 PREDICCIONES BASADAS EN LOS DATOS ENCONTRADOS")
print("="*70)

for date in ["2025-07-20", "2025-07-21"]:
    print(f"\n📅 {date}:")

    date_results = results_by_date[date]
    if date_results:
        # Mostrar los primeros 5 resultados como más probables
        print("   Números más probables:")
        shown = set()
        for i, res in enumerate(date_results[:10]):
            if res['numero'] and res['numero'] not in shown:
                shown.add(res['numero'])
                print(f"   {len(shown)}. #{res['numero']} - {res['animalito'] or ANIMALITOS.get(res['numero'], 'Desconocido')}")
                if len(shown) >= 5:
                    break
    else:
        print("   No se encontraron datos pre-cargados para esta fecha")

print("\n✅ Análisis completado.")
print("\n💡 NOTA: Estos son datos encontrados en el código fuente.")
print("Los resultados reales pueden variar.")

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
# ANALIZADOR FORENSE - CÓNDOR GANA (76 ANIMALITOS)
# Para Google Colab - Busca resultados antes de publicación

!pip install selenium webdriver-manager beautifulsoup4 requests pandas
!apt-get update
!apt-get install chromium-chromedriver

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import json
import re
from datetime import datetime
from collections import defaultdict
import pandas as pd
from bs4 import BeautifulSoup
import time
import requests

print("🎯 ANALIZADOR FORENSE - CÓNDOR GANA (76 ANIMALITOS)")
print("URL objetivo: https://tripletachira.com/resultadosh.php")
print("Fecha:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("=" * 80)

# DICCIONARIO COMPLETO DE 76 ANIMALITOS
ANIMALITOS = {
    "00": "Delfín", "0": "Ballena", "1": "Carnero", "2": "Toro", "3": "Ciempiés",
    "4": "Alacrán", "5": "León", "6": "Ranas", "7": "Perico", "8": "Ratón",
    "9": "Águila", "10": "Tigre", "11": "Gato", "12": "Caballo", "13": "Mono",
    "14": "Paloma", "15": "Zorro", "16": "Oso", "17": "Pavo", "18": "Burro",
    "19": "Chivo", "20": "Cochino", "21": "Gallo", "22": "Camello", "23": "Cebra",
    "24": "Iguana", "25": "Gallina", "26": "Vaca", "27": "Perro", "28": "Zamuro",
    "29": "Elefante", "30": "Caimán", "31": "Lapa", "32": "Ardilla", "33": "Pescado",
    "34": "Venado", "35": "Jirafa", "36": "Culebra", "37": "Tortuga", "38": "Búfalo",
    "39": "Lechuza", "40": "Avispa", "41": "Canguro", "42": "Tucán", "43": "Mariposa",
    "44": "Chigüire", "45": "Garza", "46": "Puma", "47": "Pavo Real", "48": "Puercoespín",
    "49": "Pereza", "50": "Canario", "51": "Pelícano", "52": "Pulpo", "53": "Caracol",
    "54": "Grillo", "55": "Oso Hormiguero", "56": "Tiburón", "57": "Pato", "58": "Hormiga",
    "59": "Pantera", "60": "Camaleón", "61": "Panda", "62": "Cachicamo", "63": "Cangrejo",
    "64": "Gavilán", "65": "Araña", "66": "Lobo", "67": "Avestruz", "68": "Jaguar",
    "69": "Conejo", "70": "Bisonte", "71": "Guacamaya", "72": "Gorila", "73": "Hipopótamo",
    "74": "Turpial", "75": "Guácharo"
}

# Crear diccionario inverso (nombre -> número)
ANIMALITOS_REVERSE = {}
for num, animal in ANIMALITOS.items():
    ANIMALITOS_REVERSE[animal.lower()] = num
    # Agregar variaciones sin tilde
    ANIMALITOS_REVERSE[animal.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')] = num

# Horas del Cóndor Gana
HOURS = ["09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00"]

# Configurar Chrome
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--enable-logging')
    chrome_options.add_experimental_option('perfLoggingPrefs', {'enableNetwork': True})
    chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
    return webdriver.Chrome(options=chrome_options)

# Resultados encontrados
found_results = defaultdict(list)

print("\n1️⃣ CARGANDO PÁGINA Y ANALIZANDO ESTRUCTURA...")

driver = setup_driver()
driver.get("https://tripletachira.com/resultadosh.php")
time.sleep(5)

# Obtener HTML completo
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')

print("\n2️⃣ BUSCANDO EN TODOS LOS SCRIPTS...")

scripts = soup.find_all('script')
print(f"   📜 {len(scripts)} scripts encontrados")

for i, script in enumerate(scripts):
    if script.string:
        content = script.string

        # Buscar menciones de animalitos
        for animal_name, numero in ANIMALITOS_REVERSE.items():
            if animal_name in content.lower():
                # Buscar contexto alrededor del animal
                pos = content.lower().find(animal_name)
                context = content[max(0, pos-100):pos+100]

                # Buscar hora cercana
                hour_pattern = r'(09|10|11|12|13|14|15|16|17|18):00'
                hour_match = re.search(hour_pattern, context)

                if hour_match:
                    found_results[f"{hour_match.group()}"].append({
                        'numero': numero,
                        'animal': ANIMALITOS[numero],
                        'source': f'Script #{i}',
                        'context': context
                    })
                    print(f"   ✅ Encontrado: {numero}-{ANIMALITOS[numero]} para {hour_match.group()}")

        # Buscar arrays de datos JSON
        json_patterns = [
            r'\{[^{}]*"numero"[^{}]*\}',
            r'\{[^{}]*"N"[^{}]*\}',
            r'\{[^{}]*"animal"[^{}]*\}',
            r'\[(.*?)\]'
        ]

        for pattern in json_patterns:
            matches = re.findall(pattern, content, re.DOTALL)
            for match in matches:
                try:
                    # Intentar parsear como JSON
                    if match.startswith('['):
                        data = json.loads(match)
                    else:
                        data = json.loads(f'[{match}]')

                    for item in data if isinstance(data, list) else [data]:
                        if isinstance(item, dict):
                            # Buscar número
                            numero = item.get('numero') or item.get('N') or item.get('number')
                            # Buscar animal
                            animal = item.get('animal') or item.get('S') or item.get('name')
                            # Buscar hora
                            hora = item.get('hora') or item.get('time') or item.get('H')

                            if numero and str(numero) in ANIMALITOS:
                                found_results[hora or 'Sin hora'].append({
                                    'numero': str(numero),
                                    'animal': animal or ANIMALITOS[str(numero)],
                                    'source': f'JSON en Script #{i}'
                                })
                except:
                    pass

print("\n3️⃣ EJECUTANDO ANÁLISIS JAVASCRIPT...")

# JavaScript para buscar datos en variables globales y DOM
js_analysis = """
function analyzeCondorData() {
    var results = {
        variables: {},
        dom_data: [],
        hidden_data: [],
        ajax_data: []
    };

    // Lista de animalitos para buscar
    var animalitos = ['delfín', 'ballena', 'carnero', 'toro', 'ciempiés', 'alacrán',
                      'león', 'ranas', 'perico', 'ratón', 'águila', 'tigre', 'gato',
                      'caballo', 'mono', 'paloma', 'zorro', 'oso', 'pavo', 'burro',
                      'chivo', 'cochino', 'gallo', 'camello', 'cebra', 'iguana',
                      'gallina', 'vaca', 'perro', 'zamuro', 'elefante', 'caimán',
                      'lapa', 'ardilla', 'pescado', 'venado', 'jirafa', 'culebra'];

    // 1. Buscar en variables globales
    for(let prop in window) {
        try {
            let value = window[prop];
            if(value && typeof value !== 'function') {
                let str = JSON.stringify(value).toLowerCase();

                // Buscar si contiene animalitos o números
                for(let animal of animalitos) {
                    if(str.includes(animal)) {
                        results.variables[prop] = value;
                        break;
                    }
                }

                // Buscar patrones de resultados
                if(str.includes('condor') || str.includes('resultado') ||
                   str.includes('sorteo') || /\\d{1,2}:\\d{2}/.test(str)) {
                    results.variables[prop] = value;
                }
            }
        } catch(e) {}
    }

    // 2. Buscar en el DOM
    document.querySelectorAll('*').forEach(el => {
        let text = (el.innerText || el.textContent || '').toLowerCase();

        // Buscar elementos que contengan animalitos
        for(let animal of animalitos) {
            if(text.includes(animal)) {
                // Buscar números cercanos
                let numbers = text.match(/\\b\\d{1,2}\\b/g);
                if(numbers) {
                    results.dom_data.push({
                        element: el.tagName,
                        animal: animal,
                        numbers: numbers,
                        text: text.substring(0, 200)
                    });
                }
            }
        }
    });

    // 3. Buscar elementos ocultos
    document.querySelectorAll('[style*="display:none"], [style*="visibility:hidden"], [hidden]').forEach(el => {
        let text = el.innerText || el.textContent || '';
        if(text && /\\d{1,2}/.test(text)) {
            results.hidden_data.push({
                tag: el.tagName,
                id: el.id,
                class: el.className,
                text: text.substring(0, 100)
            });
        }
    });

    // 4. Interceptar AJAX responses
    var originalXHR = window.XMLHttpRequest;
    window.XMLHttpRequest = function() {
        var xhr = new originalXHR();
        var originalOpen = xhr.open;

        xhr.open = function(method, url) {
            xhr.addEventListener('load', function() {
                if(this.responseText &&
                   (this.responseText.includes('condor') ||
                    this.responseText.match(/\\d{1,2}/))) {
                    results.ajax_data.push({
                        url: url,
                        response: this.responseText.substring(0, 500)
                    });
                }
            });
            return originalOpen.apply(this, arguments);
        };
        return xhr;
    };

    return results;
}

return analyzeCondorData();
"""

js_results = driver.execute_script(js_analysis)

print("\n📊 Resultados del análisis JavaScript:")
if js_results['variables']:
    print("   Variables encontradas:")
    for var, value in list(js_results['variables'].items())[:5]:
        print(f"      - {var}: {str(value)[:100]}")

if js_results['dom_data']:
    print("   Datos en el DOM:")
    for item in js_results['dom_data'][:5]:
        print(f"      - {item['animal']} con números {item['numbers']}")

print("\n4️⃣ PROBANDO ENDPOINTS Y PARÁMETROS...")

# Probar diferentes parámetros en la URL
test_params = [
    {'fecha': datetime.now().strftime('%Y-%m-%d')},
    {'date': datetime.now().strftime('%Y-%m-%d')},
    {'sorteo': 'condor'},
    {'juego': 'condor'},
    {'type': 'condor'},
    {'action': 'get_results'}
]

for params in test_params:
    try:
        response = requests.get("https://tripletachira.com/resultadosh.php", params=params, timeout=5)
        if response.status_code == 200:
            content = response.text

            # Buscar animalitos en la respuesta
            for animal_name, numero in ANIMALITOS_REVERSE.items():
                if animal_name in content.lower():
                    print(f"   ✅ Con parámetros {params}: encontrado {numero}-{ANIMALITOS[numero]}")
    except:
        pass

print("\n5️⃣ ANÁLISIS DE PETICIONES DE RED...")

# Obtener logs de red
logs = driver.get_log('performance')
ajax_calls = []

for log in logs:
    try:
        message = json.loads(log['message'])
        if 'Network.responseReceived' in message['message']['method']:
            response = message['message']['params']['response']
            url = response['url']

            if 'condor' in url.lower() or 'resultado' in url.lower():
                ajax_calls.append(url)
                print(f"   🌐 Petición encontrada: {url}")
    except:
        pass

print("\n6️⃣ BÚSQUEDA ESPECÍFICA POR HORAS...")

# Buscar patrones específicos por hora
hour_patterns = [
    r'(\d{1,2}):00.*?(\d{1,2})\s*[-–]\s*(\w+)',
    r'hora.*?(\d{1,2}):00.*?numero.*?(\d{1,2})',
    r'(\d{1,2}):00.*?resultado.*?(\d{1,2})',
    r'sorteo.*?(\d{1,2}):00.*?(\d{1,2})'
]

full_text = driver.execute_script("return document.body.innerText")

for pattern in hour_patterns:
    matches = re.findall(pattern, full_text, re.IGNORECASE | re.DOTALL)
    for match in matches:
        if len(match) >= 2:
            hour = f"{match[0]}:00"
            numero = match[1]

            if numero in ANIMALITOS:
                found_results[hour].append({
                    'numero': numero,
                    'animal': ANIMALITOS[numero],
                    'source': 'Patrón de texto'
                })

# Cerrar driver
driver.quit()

print("\n" + "="*80)
print("🎯 RESULTADOS ENCONTRADOS - CÓNDOR GANA")
print("="*80)

# Mostrar resultados organizados por hora
resultados_finales = []

for hour in HOURS:
    if hour in found_results and found_results[hour]:
        # Tomar el resultado más frecuente para cada hora
        numeros = [r['numero'] for r in found_results[hour]]
        from collections import Counter
        most_common = Counter(numeros).most_common(1)[0][0]

        animal = ANIMALITOS.get(most_common, "Desconocido")
        print(f"\n⏰ {hour}:")
        print(f"   Número: {most_common.zfill(2)}")
        print(f"   Animal: {animal}")
        print(f"   Fuente: {found_results[hour][0]['source']}")

        resultados_finales.append({
            'Hora': hour,
            'Número': most_common.zfill(2),
            'Animal': animal
        })
    else:
        print(f"\n⏰ {hour}:")
        print(f"   ⚠️ No se encontraron datos pre-cargados")

# Guardar resultados
if resultados_finales:
    df = pd.DataFrame(resultados_finales)
    df.to_csv('condor_resultados_forense.csv', index=False)
    print("\n📁 Resultados guardados en 'condor_resultados_forense.csv'")

print("\n📊 RESUMEN:")
print(f"   - Total de resultados encontrados: {len(resultados_finales)}")
print(f"   - Horas con datos: {', '.join([r['Hora'] for r in resultados_finales])}")

print("\n💡 NOTA: Los resultados mostrados son basados en datos encontrados en el código.")
print("Si no se encontraron todos los resultados, es posible que:")
print("- Se carguen dinámicamente más tarde")
print("- Estén encriptados")
print("- Requieran autenticación especial")

print("\n✅ Análisis forense completado.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib3-2.4.0
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InReleas

In [ ]:
# DETECTOR DE ENCRIPTACIÓN - CÓNDOR GANA
# Para Google Colab - Análisis de seguridad y encriptación

!pip install selenium webdriver-manager beautifulsoup4 cryptography pycryptodome

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import base64
import hashlib
import json
from datetime import datetime
from Crypto.Cipher import AES
import binascii

print("🔐 DETECTOR DE ENCRIPTACIÓN - CÓNDOR GANA")
print("Fecha:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("=" * 60)

# Configurar Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

print("\n1️⃣ ANALIZANDO PÁGINA...")
driver.get("https://tripletachira.com/resultadosh.php")

# JavaScript para detectar encriptación
js_detector = """
return {
    // Detectar funciones de encriptación
    crypto_functions: Object.getOwnPropertyNames(window).filter(p =>
        p.toLowerCase().includes('crypt') ||
        p.toLowerCase().includes('encode') ||
        p.toLowerCase().includes('decode') ||
        p.toLowerCase().includes('hash') ||
        p.toLowerCase().includes('cipher')
    ),

    // Detectar librerías cargadas
    loaded_scripts: Array.from(document.scripts).map(s => s.src).filter(s => s),

    // Buscar tokens o claves
    potential_keys: Object.keys(window).filter(k =>
        k.toLowerCase().includes('key') ||
        k.toLowerCase().includes('token') ||
        k.toLowerCase().includes('secret')
    ),

    // Detectar AJAX con headers especiales
    ajax_detection: (() => {
        let detected = [];
        const originalOpen = XMLHttpRequest.prototype.open;
        XMLHttpRequest.prototype.open = function() {
            detected.push({
                method: arguments[0],
                url: arguments[1]
            });
            return originalOpen.apply(this, arguments);
        };
        return detected;
    })(),

    // Buscar patterns Base64
    base64_patterns: document.body.innerHTML.match(/[A-Za-z0-9+/]{20,}={0,2}/g) || [],

    // Detectar WebSockets
    websockets: typeof WebSocket !== 'undefined',

    // LocalStorage keys
    storage_keys: Object.keys(localStorage),

    // Cookies
    cookies: document.cookie
};
"""

detection_results = driver.execute_script(js_detector)

print("\n2️⃣ RESULTADOS DE DETECCIÓN:")

# Analizar funciones crypto
if detection_results['crypto_functions']:
    print("\n🔑 Funciones de encriptación detectadas:")
    for func in detection_results['crypto_functions']:
        print(f"   - {func}")

# Analizar scripts externos
print("\n📦 Librerías externas:")
crypto_libs = ['crypto', 'aes', 'rsa', 'md5', 'sha', 'bcrypt']
for script in detection_results['loaded_scripts']:
    for lib in crypto_libs:
        if lib in script.lower():
            print(f"   ⚠️ {script}")

# Analizar Base64
if detection_results['base64_patterns']:
    print(f"\n🔤 Posibles datos Base64: {len(detection_results['base64_patterns'])} encontrados")
    for b64 in detection_results['base64_patterns'][:3]:
        try:
            decoded = base64.b64decode(b64)
            if any(c < 32 or c > 126 for c in decoded[:20]):
                print(f"   - Binario encriptado: {b64[:30]}...")
            else:
                print(f"   - Texto decodificado: {decoded[:30]}")
        except:
            pass

# Analizar tokens/keys
if detection_results['potential_keys']:
    print("\n🗝️ Posibles claves/tokens:")
    for key in detection_results['potential_keys']:
        print(f"   - {key}")

# Detectar tipo de encriptación
print("\n3️⃣ ANÁLISIS DE ENCRIPTACIÓN:")

# Buscar patrones específicos
html_source = driver.page_source

encryption_patterns = {
    'AES': [r'AES', r'aes', r'CryptoJS\.AES', r'iv\s*[:=]', r'cipher'],
    'RSA': [r'RSA', r'rsa', r'publicKey', r'privateKey'],
    'JWT': [r'eyJ[A-Za-z0-9_-]+\.eyJ[A-Za-z0-9_-]+\.[A-Za-z0-9_-]+'],
    'MD5': [r'md5', r'MD5', r'[a-f0-9]{32}'],
    'SHA': [r'sha\d+', r'SHA\d+', r'[a-f0-9]{40,64}'],
    'Base64': [r'btoa', r'atob', r'base64'],
    'Custom': [r'encrypt', r'decrypt', r'encode', r'decode']
}

detected_encryption = []
for enc_type, patterns in encryption_patterns.items():
    for pattern in patterns:
        if re.search(pattern, html_source):
            detected_encryption.append(enc_type)
            break

print(f"Tipos detectados: {', '.join(set(detected_encryption)) if detected_encryption else 'Ninguno'}")

# Buscar autenticación
auth_patterns = [
    r'authorization\s*[:=]',
    r'bearer\s+[A-Za-z0-9_-]+',
    r'api[_-]?key',
    r'X-[A-Za-z-]+:\s*\S+',
    r'authenticate',
    r'login'
]

auth_detected = any(re.search(p, html_source, re.I) for p in auth_patterns)
print(f"\n🔐 Autenticación requerida: {'SÍ' if auth_detected else 'NO'}")

# Detectar protección temporal
time_patterns = [
    r'timestamp',
    r'expires',
    r'valid[_-]?until',
    r'Date\.now\(\)',
    r'new Date\(\)'
]

time_protection = any(re.search(p, html_source, re.I) for p in time_patterns)
print(f"⏰ Protección temporal: {'SÍ' if time_protection else 'NO'}")

print("\n4️⃣ RECOMENDACIONES:")

if 'AES' in detected_encryption:
    print("\n💡 ENCRIPTACIÓN AES DETECTADA:")
    print("   - Buscar la clave (key) y vector de inicialización (iv)")
    print("   - Revisar archivos JS para encontrar: CryptoJS.AES.decrypt()")
    print("   - La clave podría estar en localStorage o como variable global")

if 'JWT' in detected_encryption:
    print("\n💡 JWT TOKENS DETECTADOS:")
    print("   - Los tokens JWT se pueden decodificar en jwt.io")
    print("   - Buscar en headers: Authorization: Bearer <token>")

if auth_detected:
    print("\n💡 AUTENTICACIÓN DETECTADA:")
    print("   - Inspeccionar cookies de sesión")
    print("   - Buscar endpoints de login")
    print("   - Revisar headers HTTP requeridos")

if time_protection:
    print("\n💡 PROTECCIÓN TEMPORAL:")
    print("   - Los datos pueden cargarse en tiempo real")
    print("   - Verificar si hay polling o WebSockets")

print("\n5️⃣ TÉCNICAS SUGERIDAS:")
print("\n🛠️ Para bypasear:")
print("1. Interceptar respuestas AJAX con Proxy")
print("2. Modificar JavaScript: delete window.encryptFunction")
print("3. Sobrescribir funciones: window.btoa = (x) => x")
print("4. Inyectar script antes de carga:")
print("   driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {")
print("       'source': 'window.decrypt = (x) => x;'")
print("   })")

# Intento de bypass básico
print("\n6️⃣ INTENTANDO BYPASS BÁSICO...")

bypass_script = """
// Sobrescribir funciones de encriptación comunes
if(window.CryptoJS) window.CryptoJS.AES.decrypt = (x) => ({toString: () => x});
if(window.btoa) window.btoa = (x) => x;
if(window.encrypt) window.encrypt = (x) => x;

// Interceptar localStorage
const originalSetItem = localStorage.setItem;
localStorage.setItem = function(key, value) {
    console.log('LocalStorage:', key, '=', value);
    return originalSetItem.apply(this, arguments);
};

// Retornar datos interceptados
return {
    intercepted: window._interceptedData || null,
    decrypted: window._decryptedResults || null
};
"""

bypass_result = driver.execute_script(bypass_script)
if bypass_result['intercepted'] or bypass_result['decrypted']:
    print("   ✅ Datos interceptados:", bypass_result)
else:
    print("   ❌ No se pudieron interceptar datos")

driver.quit()

print("\n" + "="*60)
print("✅ Análisis completado")
print("\n📌 RESUMEN FINAL:")
print(f"   - Encriptación: {', '.join(set(detected_encryption)) if detected_encryption else 'No detectada'}")
print(f"   - Autenticación: {'Requerida' if auth_detected else 'No requerida'}")
print(f"   - Protección temporal: {'Sí' if time_protection else 'No'}")
print("\n⚠️ NOTA: Este análisis es con fines educativos únicamente.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.0 MB/s eta 0:00:00
🔐 DETECTOR DE ENCRIPTACIÓN - CÓNDOR GANA
Fecha: 2025-07-21 16:38:02

1️⃣ ANALIZANDO PÁGINA...

2️⃣ RESULTADOS DE DETECCIÓN:

🔑 Funciones de encriptación detectadas:
   - decodeURI
   - decodeURIComponent
   - encodeURI
   - encodeURIComponent
   - TextEncoderStream
   - TextEncoder
   - TextDecoderStream
   - TextDecoder
   - RTCEncodedVideoFrame
   - RTCEncodedAudioFrame
   - MediaEncryptedEvent
   - HashChangeEvent
   - EncodedVideoChunk
   - EncodedAudioChunk
   - Crypto
   - crypto
   - onhashchange
   - AudioDecoder
   - AudioEncoder
   - CryptoKey
   - GPUCommandEncoder
   - GPUComputePassEncoder
   - GPURenderBundleEncoder
   - GPURenderPassEncoder
   - ImageDecoder
   - SubtleCrypto
   - VideoDecoder
   - VideoEncoder

📦 Librerías externas:

🗝️ Posibles claves/tokens:
   - onkeydown
   - onkeypress
   - onkeyup

3️⃣ ANÁLISIS DE ENCRIPTACIÓN:
Tipos detectados: AES

🔐 Autenticación requerida: NO
⏰ Protec

In [4]:
# DESENCRIPTADOR AES - CÓNDOR GANA
# Para Google Colab - Intenta desencriptar datos AES

!pip install selenium webdriver-manager beautifulsoup4 pycryptodome requests
!apt-get update
!apt-get install chromium-chromedriver

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import json
import re
import base64
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
import hashlib
from datetime import datetime
import time

print("🔓 DESENCRIPTADOR AES - CÓNDOR GANA")
print("Fecha:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("=" * 80)

# Configurar Chrome con capacidades de intercepción
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--enable-logging')
    chrome_options.add_experimental_option('perfLoggingPrefs', {'enableNetwork': True})
    chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
    return webdriver.Chrome(options=chrome_options)

# Funciones de desencriptación AES
def try_aes_decrypt(encrypted_data, key, iv=None):
    """Intenta desencriptar datos AES"""
    try:
        # Si la clave es string, convertir a bytes
        if isinstance(key, str):
            key = key.encode('utf-8')
            # Ajustar longitud de clave (AES requiere 16, 24 o 32 bytes)
            if len(key) < 16:
                key = key.ljust(16, b'\0')
            elif len(key) > 16 and len(key) < 24:
                key = key[:16]
            elif len(key) > 24 and len(key) < 32:
                key = key[:24]
            elif len(key) > 32:
                key = key[:32]

        # Decodificar datos si están en base64
        if isinstance(encrypted_data, str):
            try:
                encrypted_data = base64.b64decode(encrypted_data)
            except:
                pass

        # Modo CBC con IV
        if iv:
            if isinstance(iv, str):
                iv = iv.encode('utf-8')[:16]
            cipher = AES.new(key, AES.MODE_CBC, iv)
        else:
            # Modo ECB sin IV
            cipher = AES.new(key, AES.MODE_ECB)

        decrypted = cipher.decrypt(encrypted_data)

        # Intentar quitar padding
        try:
            decrypted = unpad(decrypted, AES.block_size)
        except:
            pass

        # Intentar decodificar como texto
        return decrypted.decode('utf-8', errors='ignore')

    except Exception as e:
        return None

print("\n1️⃣ CARGANDO PÁGINA Y BUSCANDO CLAVES AES...")

driver = setup_driver()

# Script para interceptar y buscar claves
intercept_script = """
// Interceptar CryptoJS si existe
window.interceptedKeys = [];
window.interceptedData = [];

// Sobrescribir funciones de encriptación/desencriptación
if(typeof CryptoJS !== 'undefined') {
    // Interceptar AES
    var originalDecrypt = CryptoJS.AES.decrypt;
    CryptoJS.AES.decrypt = function(ciphertext, key, options) {
        window.interceptedKeys.push({
            type: 'AES',
            key: key,
            options: options,
            ciphertext: ciphertext.toString()
        });
        return originalDecrypt.apply(this, arguments);
    };

    var originalEncrypt = CryptoJS.AES.encrypt;
    CryptoJS.AES.encrypt = function(message, key, options) {
        window.interceptedKeys.push({
            type: 'AES',
            key: key,
            options: options,
            plaintext: message.toString()
        });
        return originalEncrypt.apply(this, arguments);
    };
}

// Interceptar localStorage
var originalSetItem = Storage.prototype.setItem;
Storage.prototype.setItem = function(key, value) {
    window.interceptedData.push({
        type: 'localStorage',
        key: key,
        value: value
    });
    return originalSetItem.apply(this, arguments);
};

// Buscar variables globales con posibles claves
window.foundKeys = [];
for(let prop in window) {
    if(prop.toLowerCase().includes('key') ||
       prop.toLowerCase().includes('secret') ||
       prop.toLowerCase().includes('token') ||
       prop.toLowerCase().includes('pass')) {
        try {
            window.foundKeys.push({
                name: prop,
                value: window[prop]
            });
        } catch(e) {}
    }
}

console.log('Interceptores instalados');
"""

# Cargar página principal
driver.get("https://tripletachira.com/condor.php")
driver.execute_script(intercept_script)
time.sleep(3)

# Cargar página de resultados
driver.get("https://tripletachira.com/resultadosh.php")
time.sleep(3)

print("\n2️⃣ BUSCANDO CLAVES Y DATOS ENCRIPTADOS...")

# Buscar en el código fuente
page_source = driver.page_source

# Patrones de claves comunes
key_patterns = [
    r'["\']key["\']\s*:\s*["\']([^"\']+)["\']',
    r'["\']secret["\']\s*:\s*["\']([^"\']+)["\']',
    r'["\']password["\']\s*:\s*["\']([^"\']+)["\']',
    r'AES\.decrypt\([^,]+,\s*["\']([^"\']+)["\']',
    r'var\s+\w*key\w*\s*=\s*["\']([^"\']+)["\']',
    r'const\s+\w*key\w*\s*=\s*["\']([^"\']+)["\']'
]

potential_keys = []

for pattern in key_patterns:
    matches = re.findall(pattern, page_source, re.IGNORECASE)
    potential_keys.extend(matches)

# Buscar datos encriptados
encrypted_patterns = [
    r'["\']([A-Za-z0-9+/]{32,}={0,2})["\']',  # Base64
    r'["\']([0-9a-fA-F]{32,})["\']',          # Hex
]

potential_encrypted = []

for pattern in encrypted_patterns:
    matches = re.findall(pattern, page_source)
    potential_encrypted.extend(matches[:20])  # Limitar a 20

print(f"\n🔑 Claves potenciales encontradas: {len(potential_keys)}")
for key in potential_keys[:5]:
    print(f"   - {key[:20]}...")

print(f"\n🔐 Datos encriptados potenciales: {len(potential_encrypted)}")

# Obtener datos interceptados
intercepted = driver.execute_script("return {keys: window.interceptedKeys || [], data: window.interceptedData || [], found: window.foundKeys || []}")

if intercepted['keys']:
    print("\n✅ CLAVES INTERCEPTADAS EN TIEMPO REAL:")
    for item in intercepted['keys']:
        print(f"   - Tipo: {item['type']}, Clave: {str(item.get('key', ''))[:30]}")

print("\n3️⃣ INTENTANDO DESENCRIPTAR DATOS...")

# Claves comunes a probar
common_keys = [
    "tripletachira",
    "condorgana",
    "condor2025",
    "tachira2025",
    "resultado",
    "sorteo",
    "123456789012345",  # 16 bytes
    "12345678901234567890123456789012",  # 32 bytes
]

# Agregar claves encontradas
all_keys = list(set(potential_keys + common_keys))

# También buscar claves basadas en fecha
today = datetime.now()
date_keys = [
    today.strftime("%Y%m%d"),
    today.strftime("%Y-%m-%d"),
    today.strftime("%d%m%Y"),
    "condor" + today.strftime("%Y%m%d"),
]
all_keys.extend(date_keys)

# Intentar desencriptar
decrypted_results = []

for encrypted in potential_encrypted[:10]:  # Probar con los primeros 10
    for key in all_keys:
        # Probar sin IV
        result = try_aes_decrypt(encrypted, key)
        if result and len(result) > 5 and not all(c == '\0' for c in result):
            # Verificar si contiene números o palabras relevantes
            if re.search(r'\d{1,2}', result) or 'condor' in result.lower():
                decrypted_results.append({
                    'encrypted': encrypted[:30] + '...',
                    'key': key,
                    'decrypted': result[:200],
                    'iv': 'None'
                })
                print(f"\n✅ DESENCRIPTADO:")
                print(f"   Clave: {key}")
                print(f"   Resultado: {result[:100]}")

        # Probar con IV común (16 ceros)
        result = try_aes_decrypt(encrypted, key, b'\0' * 16)
        if result and len(result) > 5:
            if re.search(r'\d{1,2}', result):
                decrypted_results.append({
                    'encrypted': encrypted[:30] + '...',
                    'key': key,
                    'decrypted': result[:200],
                    'iv': 'zeros'
                })

print("\n4️⃣ BUSCANDO EN RESPUESTAS AJAX...")

# Ejecutar peticiones AJAX
ajax_script = """
var results = [];

// Hacer peticiones a endpoints conocidos
var endpoints = [
    'resultadosh.php',
    'condor.php',
    'nadacondor.php',
    'api/condor',
    'ajax/results'
];

var promises = endpoints.map(endpoint => {
    return fetch(location.origin + '/' + endpoint, {
        method: 'POST',
        headers: {'Content-Type': 'application/x-www-form-urlencoded'},
        body: 'dataj=' + new Date().toISOString().split('T')[0]
    })
    .then(r => r.text())
    .then(text => ({endpoint: endpoint, response: text}))
    .catch(e => ({endpoint: endpoint, error: e.toString()}));
});

return Promise.all(promises);
"""

ajax_results = driver.execute_script(ajax_script)
time.sleep(3)

# Obtener resultados AJAX
ajax_data = driver.execute_script("return arguments[0]", ajax_results)

if ajax_data:
    print("\n📡 Respuestas AJAX:")
    for item in ajax_data:
        if 'response' in item and item['response']:
            print(f"\n   Endpoint: {item['endpoint']}")
            # Buscar datos encriptados en respuesta
            encrypted_in_response = re.findall(r'[A-Za-z0-9+/]{32,}={0,2}', item['response'])
            if encrypted_in_response:
                print(f"   Datos encriptados encontrados: {len(encrypted_in_response)}")

                # Intentar desencriptar
                for enc in encrypted_in_response[:3]:
                    for key in all_keys[:5]:
                        result = try_aes_decrypt(enc, key)
                        if result and re.search(r'\d{1,2}', result):
                            print(f"   ✅ Desencriptado con {key}: {result[:50]}")

print("\n5️⃣ ANÁLISIS FINAL DE RESULTADOS...")

# Buscar patrones en los datos desencriptados
if decrypted_results:
    print("\n🎯 DATOS DESENCRIPTADOS ENCONTRADOS:")

    for item in decrypted_results:
        text = item['decrypted']

        # Buscar números del 0-75 (animalitos)
        numbers = re.findall(r'\b([0-7]?[0-9])\b', text)
        valid_numbers = [n for n in numbers if int(n) <= 75]

        if valid_numbers:
            print(f"\n   Números encontrados: {valid_numbers}")

            # Buscar horas
            hours = re.findall(r'(09|10|11|12|13|14|15|16|17|18):00', text)
            if hours:
                print(f"   Horas: {hours}")

driver.quit()

print("\n" + "="*80)
print("📊 RESUMEN DE DESENCRIPTACIÓN")
print("="*80)

print(f"\n- Claves probadas: {len(all_keys)}")
print(f"- Datos encriptados analizados: {len(potential_encrypted)}")
print(f"- Desencriptaciones exitosas: {len(decrypted_results)}")

if decrypted_results:
    print("\n✅ SE LOGRARON DESENCRIPTAR DATOS")
    print("Los resultados desencriptados se muestran arriba")
else:
    print("\n❌ NO SE LOGRARON DESENCRIPTAR DATOS")
    print("\nPosibles razones:")
    print("- La clave no está en el código cliente")
    print("- Usan un método de encriptación diferente")
    print("- Los datos se encriptan en el servidor")
    print("- Necesitan autenticación especial")

print("\n💡 PRÓXIMOS PASOS:")
print("1. Monitorear el tráfico de red con mitmproxy")
print("2. Analizar el código JavaScript minificado")
print("3. Buscar en archivos .js externos")
print("4. Intentar con más variaciones de claves")

print("\n✅ Análisis completado.")

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,632 B in 1s (3,336 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... D